# Volumetric Heat-Exchange Source Term Verification

This Jupyter notebook documents and verifies the implementation of **echange\_thermique\_volumique**.  
The demonstration problem is a simplified **plate heat exchanger** made of three coupled domains:

1. **Primary fluid channel** – hot, single-phase thermo-hydraulic flow (heat donor)  
2. **Steel plates** – solid conduction domain separating the two fluids  
3. **Secondary fluid channel** – cold, single-phase thermo-hydraulic flow (heat receiver)

In [ ]:
from trustutils.jupyter import run

run.introduction("Yannick Gorsse")
run.TRUST_parameters()

In [ ]:
run.reset()
run.initBuildDirectory()

Lx = 0.5
Ly = 1
ep_plaque = 1e-3
ep_sec  = 5e-3
ep_prim = 5e-3
np = Lx / (2 * ep_plaque + ep_sec + ep_prim)
phi1 = (np * ep_prim) / Lx
phic = (2 * np * ep_plaque) / Lx
phi2 = 1 - phi1 - phic
ai1 = 2 * np / Lx
ai2 = 2 * np / Lx
dh1 = 2 * ep_prim
dh2 = 2 * ep_sec


subs = {"phi1" : phi1, "phic" : phic, "phi2" : phi2, "ai1" : ai1, "ai2" : ai2, "dh1" : dh1, "dh2" : dh2, "epaisseur_acier" : ep_plaque}

dis = [("PolyMAC_P0", "-"), ("PolyMAC_P0P1NC", "-x"), ("VDF", "-+")]
# single domain case
for d, _ in dis:
    tc = run.addCaseFromTemplate("jdd.data", d, {**subs, **{"dis": d}})

run.runCases()
run.tablePerf()

In [ ]:
from trustutils import plot

a = plot.Graph("Residuals")
for d, m in dis:
    a.addResidu(f"{run.BUILD_DIRECTORY}/{d}/jdd.dt_ev", label=d, marker=m)

a.scale(yscale="log")

In [ ]:
a = plot.Graph("Temperatures")

for d, m in dis:
    for c in ["PRIM", "SOL", "SEC"]:
        a.addSegment(f"{run.BUILD_DIRECTORY}/{d}/jdd_T{c}.son", label=f"{d}, {c}", marker=m)
